In [ ]:
!pip install -q tensorflow==2.0.0-beta1

     |████████████████████████████████| 87.9MB 359kB/s 
     |████████████████████████████████| 501kB 43.2MB/s 
     |████████████████████████████████| 3.1MB 34.2MB/s 


In [2]:
import tensorflow as tf
import unicodedata
import string
import numpy as np
import re
import matplotlib.pyplot as plt
# from google.colab import drive

In [ ]:
# drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
keras = tf.keras
print(tf.__version__)

2.6.1


In [4]:
!wget https://www.manythings.org/anki/fra-eng.zip
!unzip  fra-eng.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
SOS_token = 0
EOS_token = 1

In [6]:
class Lang(object):
    def __init__(self, name):
        self.name = name
        self.word2int = {}
        self.word2count = {}
        self.int2word = {0 : "SOS", 1 : "EOS"}
        self.n_words = 2
        
    def addWord(self, word):
        if word not in self.word2int:
            self.word2int[word] = self.n_words
            self.word2count[word] = 1
            self.int2word[self.n_words] = word
            self.n_words += 1
            
        else:
            self.word2count[word] += 1
    
    def addSentence(self, sentence):
        for word in sentence.split(" "):
            self.addWord(word)

In [7]:
def unicodeToAscii(s):
    return "".join(c for c in unicodedata.normalize("NFD", s) \
                   if unicodedata.category(c) != "Mn")

In [8]:
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    
    s = re.sub(r"([!.?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z?.!]+", " ", s)
    return s

In [49]:
def load_dataset():
    with open("fra.txt",'r') as f:
        lines = f.readlines()

    # pairs = [[normalizeString(pair) for pair in
    #           line.strip().split('\t')] for line in lines]
    pairs = []
    for x in lines:
        pair = x.split('\t')
        pairs.append([pair[0], pair[1]])
    return pairs

In [50]:
pairs = load_dataset()
pairs[856:858]

[['Calm down.', 'Du calme.'], ['Calm down.', 'Tranquille.']]

In [51]:
pairs

[['Go.', 'Va !'],
 ['Go.', 'Marche.'],
 ['Go.', 'Bouge !'],
 ['Hi.', 'Salut !'],
 ['Hi.', 'Salut.'],
 ['Run!', 'Coursâ€¯!'],
 ['Run!', 'Courezâ€¯!'],
 ['Run!', 'Prenez vos jambes Ã\xa0 vos cous !'],
 ['Run!', 'File !'],
 ['Run!', 'Filez !'],
 ['Run!', 'Cours !'],
 ['Run!', 'Fuyez !'],
 ['Run!', 'Fuyons !'],
 ['Run.', 'Coursâ€¯!'],
 ['Run.', 'Courezâ€¯!'],
 ['Run.', 'Prenez vos jambes Ã\xa0 vos cous !'],
 ['Run.', 'File !'],
 ['Run.', 'Filez !'],
 ['Run.', 'Cours !'],
 ['Run.', 'Fuyez !'],
 ['Run.', 'Fuyons !'],
 ['Who?', 'Qui ?'],
 ['Wow!', 'Ã‡a alorsâ€¯!'],
 ['Duck!', 'Ã€ terreÂ\xa0!'],
 ['Duck!', 'Baisse-toiÂ\xa0!'],
 ['Duck!', 'Baissez-vousÂ\xa0!'],
 ['Fire!', 'Au feu !'],
 ['Help!', "Ã€ l'aideâ€¯!"],
 ['Hide.', 'Cache-toi.'],
 ['Hide.', 'Cachez-vous.'],
 ['Jump!', 'Saute.'],
 ['Jump.', 'Saute.'],
 ['Stop!', 'Ã‡a suffitâ€¯!'],
 ['Stop!', 'Stopâ€¯!'],
 ['Stop!', 'ArrÃªte-toi !'],
 ['Wait!', 'Attends !'],
 ['Wait!', 'Attendez !'],
 ['Wait!', 'Attendez.'],
 ['Wait.', 'Attends !'],
 ['W

In [56]:
MAX_LENGTH = 10
def sentencetoIndexes(sentence, lang):
    indexes = [lang.word2int[word] for word in sentence.split()]
    indexes.append(EOS_token)
    return indexes

def filterPair(p):
    return len(p[0].split()) < MAX_LENGTH and len(p[1].split()) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

pairs = filterPairs(pairs)

In [57]:
pairs[0]

['Go.', 'Va !']

In [58]:
def build_lang(lang1, lang2, max_length=10):
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    input_seq = []
    output_seq = []
    
    for pair in pairs:
        input_lang.addSentence(pair[1])
        output_lang.addSentence(pair[0])
    for pair in pairs:
        input_seq.append(sentencetoIndexes(pair[1], input_lang))
        output_seq.append(sentencetoIndexes(pair[0], output_lang))
    return keras.preprocessing.sequence.pad_sequences(input_seq, maxlen=max_length, padding='post',
                                                      truncating='post'), \
keras.preprocessing.sequence.pad_sequences(output_seq, padding='post', truncating='post'), input_lang, output_lang

In [59]:
input_tensor, output_tensor, input_lang, output_lang = build_lang('fr', 'en')

KeyError: 'Ã'

In [17]:
input_lang

In [ ]:
input_lang.name

'fr'

In [18]:
BATCH_SIZE = 16
BUFFER_SIZE = len(input_tensor)
dataset = tf.data.Dataset.from_tensor_slices((input_tensor, output_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)

In [19]:
class Encoder(keras.models.Model):
    def __init__(self, vocab_size, num_hidden=256, num_embedding=256, batch_size=16):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.num_hidden = num_hidden
        self.num_embedding = num_embedding
        self.embedding = keras.layers.Embedding(vocab_size, num_embedding)
        self.gru = keras.layers.GRU(num_hidden, return_sequences=True,
                                    recurrent_initializer='glorot_uniform',
                                   return_state=True)
        
    def call(self, x, hidden):
        embedded = self.embedding(x)
        rnn_out, hidden = self.gru(embedded, initial_state=hidden)
        return rnn_out, hidden
    def init_hidden(self):
        return tf.zeros(shape=(self.batch_size, self.num_hidden))

In [20]:
inputs, outputs = next(iter(dataset))
hidden = tf.zeros((16, 256))

In [21]:
encoder = Encoder(input_lang.n_words)

In [22]:
e_outputs, e_hidden = encoder(inputs, hidden)

In [23]:
e_hidden

<tf.Tensor: shape=(16, 256), dtype=float32, numpy=
array([[-0.01316416, -0.01187541,  0.03886399, ..., -0.02306043,
         0.0013533 , -0.01424476],
       [-0.01347641, -0.01399617,  0.04546529, ..., -0.02501968,
         0.00277372, -0.01384097],
       [-0.01612211, -0.00936785,  0.03525378, ..., -0.01767884,
         0.0026657 , -0.01332709],
       ...,
       [-0.01279999, -0.01188518,  0.04017877, ..., -0.01974327,
         0.00475456, -0.01134078],
       [-0.01489186, -0.01252952,  0.04177019, ..., -0.02204917,
         0.00012073, -0.01244862],
       [-0.01376239, -0.01104986,  0.04156855, ..., -0.02214067,
         0.00313638, -0.0132139 ]], dtype=float32)>

In [24]:
class BahdanauAttention(keras.models.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
    
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
        
    def call(self, encoder_out, hidden):
        #shape of encoder_out : batch_size, seq_length, hidden_dim 
        #shape of encoder_hidden : batch_size, hidden_dim 
        
        hidden = tf.expand_dims(hidden, axis=1) #out:
        
        score = self.V(tf.nn.tanh(self.W1(encoder_out) + \
                                  self.W2(hidden))) 
        
        attn_weights = tf.nn.softmax(score, axis=1)
        
        context =  attn_weights * encoder_out 
        context = tf.reduce_sum(context, axis=1)
        return context, attn_weights
        

In [25]:
attn = BahdanauAttention(256)

In [26]:
context, attn_weights = attn(e_outputs, e_hidden)

In [27]:
attn_weights.shape

TensorShape([16, 10, 1])

In [28]:
class Decoder(keras.models.Model):
    def __init__(self, vocab_size, dec_dim=256, embedding_dim=256):
        super(Decoder, self).__init__()
        
        self.attn = BahdanauAttention(dec_dim)
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(dec_dim, recurrent_initializer='glorot_uniform',
                                   return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        
    def call(self, x, enc_hidden, enc_out):
        x = self.embedding(x)
        context, attn_weights = self.attn(enc_out, enc_hidden)
        x = tf.concat((tf.expand_dims(context, 1), x), -1)
        r_out, hidden = self.gru(x, initial_state=enc_hidden)
        out = tf.reshape(r_out,shape=(-1, r_out.shape[2]))
        return self.fc(out), hidden, attn_weights

In [29]:
decoder = Decoder(output_lang.n_words)

In [30]:
input_tensor, output_tensor = next(iter(dataset))

In [31]:
x = np.expand_dims(output_tensor[:,1], -1)

In [32]:
def loss_fn(real, pred):
    criterion = keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                           reduction='none')
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    _loss = criterion(real, pred)
    mask = tf.cast(mask, dtype=_loss.dtype)
    _loss *= mask
    return tf.reduce_mean(_loss)

In [33]:
optimizer = tf.optimizers.Adam()

In [34]:
def train_step(input_tensor, target_tensor, enc_hidden):
    loss = 0.0
    with tf.GradientTape() as tape:
    
        batch_size = input_tensor.shape[0]
        enc_output, enc_hidden = encoder(input_tensor, enc_hidden)

        SOS_tensor = np.array([SOS_token])
        dec_input = tf.squeeze(tf.expand_dims([SOS_tensor]*batch_size, 1), -1)
        dec_hidden = enc_hidden

        for tx in range(target_tensor.shape[1]-1):
          
            dec_out, dec_hidden, _ = decoder(dec_input, dec_hidden,
                                            enc_output)
            loss += loss_fn(target_tensor[:, tx], dec_out)
            dec_input = tf.expand_dims(target_tensor[:, tx], 1)

    batch_loss = loss / target_tensor.shape[1]
    t_variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, t_variables)
    optimizer.apply_gradients(zip(gradients, t_variables))
    return batch_loss
    

In [35]:
hidden = tf.zeros(shape=(16, 256))
loss = train_step(input_tensor, output_tensor, hidden)
print(loss)

tf.Tensor(7.0363297, shape=(), dtype=float32)


In [36]:
def checkpoint(model, name=None):
    if name is not None:
        model.save_weights('/content/gdrive/My Drive/{}.h5'.format(name))
    else:
        raise NotImplementedError

In [38]:
EPOCHS = 10
log_every = 50
steps_per_epoch = len(pairs) // BATCH_SIZE
loss_list = []

for e in range(1, EPOCHS):
    
    total_loss = 0.0
    enc_hidden = encoder.init_hidden()
    
    for idx, (input_tensor, target_tensor) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(input_tensor, target_tensor, hidden)
        total_loss += batch_loss
        
        if idx % log_every == 0:
            loss_list.append(batch_loss)
            print("Epochs: {} batch_loss: {:.4f}".format(e, batch_loss))
            checkpoint(encoder, 'encoder')
            checkpoint(decoder, 'decoder')
            
    if e % 2 == 0:
        print("Epochs: {}/{} total_loss: {:.4f}".format(
        e, EPOCHS, total_loss / steps_per_epoch))

Epochs: 1 batch_loss: 6.7956


OSError: Unable to create file (unable to open file: name = '/content/gdrive/My Drive/encoder.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [ ]:
def translate(sentence, max_length=10):
    result = ''
    attention_plot = np.zeros((10,10))
    sentence = normalizeString(sentence)
    sentence = sentencetoIndexes(sentence, input_lang)
    sentence = keras.preprocessing.sequence.pad_sequences([sentence],padding='post',
                                                      maxlen=max_length, truncating='post')
    
    encoder_hidden = hidden = [tf.zeros((1, 256))]
    
    enc_out, enc_hidden = encoder(sentence, encoder_hidden)
    
    dec_hidden = enc_hidden
    SOS_tensor = np.array([SOS_token])
    dec_input = tf.squeeze(tf.expand_dims([SOS_tensor], 1), -1)
    
    for tx in range(max_length):
        dec_out, dec_hidden, attn_weights = decoder(dec_input,
                                                   dec_hidden, enc_out)
        attn_weights = tf.reshape(attn_weights, (-1, ))
        attention_plot[tx] = attn_weights.numpy()
        pred = tf.argmax(dec_out, axis=1).numpy()
        result += output_lang.int2word[pred[0]] + " "
        if output_lang.int2word[pred[0]] == "EOS":
            break
        dec_input = tf.expand_dims(pred, axis=1)
    return result, attention_plot

In [ ]:
sentence = "j'ai besoin de quelqu'un pour m'aider ?"
pred, attn_weights = translate(sentence)
print(pred)

i need somebody to help me ? EOS 


In [ ]:
def plot_attention(attention, sentence, predicted_sentence):
    sentence = normalizeString(sentence)
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
from matplotlib import ticker

In [ ]:
attn_weights = attn_weights[:len(pred.split(' ')), :len(sentence.split(' '))]
plot_attention(attn_weights, sentence.split(), pred.split())

AttributeError: ignored

In [ ]:
encoder.load_weights('/content/gdrive/My Drive/encoder.h5')
decoder.load_weights('/content/gdrive/My Drive/decoder.h5')